In [ ]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import pylab as plot

a = 'ML_ift+relax+796.csv'
data = np.genfromtxt(a, dtype = None, delimiter = ',', skip_header = 0, filling_values = 0, invalid_raise = False, encoding = 'latin-1')
[Row,Column] = data.shape
print(data[0:2,:])

In [ ]:
newdataset = np.asarray([data[:,2]*100,np.round(data[:,0]*3600/6.5,0),data[:,4],data[:,5],data[:,6],data[:,7],data[:,8]]).T
print("newdataset = ", newdataset[0:2,:])
np.savetxt(r'C:\Users\Chun-Po\Documents\Python Scripts\儲存分析結果(數據)\改掉SOH定義的資料集.csv',newdataset,fmt='%s',delimiter=',')

In [ ]:
'''排除nan跟outliers'''

filename = r"C:\Users\Chun-Po\Documents\Python Scripts\儲存分析結果(數據)\改掉SOH定義的資料集.csv"     # print(data)
data = np.genfromtxt(filename, delimiter = ',', dtype = None, skip_header = 0, encoding = 'UTF-8')     #, usecols = [7,15,37,38,53]
[Row,Column] = data.shape     # 確認data(?)...print(data.shape); print(data); print(type(data[1,0]))
err_Chrtime = [119, 934, 1062, 1341, 1354, 1467, 1858, 2097, 2220, 2227, 2243, 2479, 3376, 3484, 3485, 3750, 4294, 4375, 4383, 4873, 5324, 5766, 6918, 6935, 7119, 7611, 8363, 8955, 9120, 9515, 10055, 10083, 10177, 10191, 10342, 10390, 11527, 12168, 12849, 13317, 13557, 13853, 14121, 14222, 14618, 16676, 17079, 18105, 18193, 18584, 18626, 18814, 19038, 19107, 19287, 19527, 19663, 19669, 20254, 20742, 21440, 21614, 21719, 21804, 21961, 21963, 23175, 23348, 23948, 24152, 24388, 24397, 24410, 25981, 26001] 
err_3mrelax = [rex for rex, rexx in enumerate(data[:,2]) if rexx>0]     # =[[119, 1062, 1341, 1467, 1858, 2220, 2227, 2243, 2479, 2900, 3484, 3485, 3750, 3823, 4375, 4383, 5324, 5766, 6935, 7119, 8363, 8955, 9120, 10083, 10177, 10191, 11527, 12849, 13557, 13853, 14222, 14618, 17079, 18193, 18584, 18626, 18814, 19038, 19107, 19287, 19527, 19663, 19669, 20254, 20742, 21614, 21719, 21804, 21963, 22906, 23175, 23348, 23948, 24397, 24410, 25981, 26001]
err_inf2500 = [ift for ift, iftt in enumerate(data[:,3]) if iftt == 2500 or iftt == 0]
err_vot_nan = np.argwhere(np.isnan(data[:,3:6]));     # print("err_vot_nan = ", err_vot_nan); print(err_vot_nan.shape); print(type(err_vot_nan[0]))
err_all_nan = []
for i in range(len(err_vot_nan)):
    errfornan = err_vot_nan[i][0].astype(np.int64)
    err_all_nan.append(errfornan)     # print("err_all_nan = ",err_all_nan); print(len(err_all_nan))
err_VPVD = [vpv for vpv, vpvp in enumerate(data[:,6]) if vpvp == 0]
print("removed    VPVD = ", len(err_VPVD))
err = sorted(set(err_Chrtime+err_3mrelax+err_inf2500+err_all_nan+err_VPVD))     # err_V15_nan+err_VPVD_nan；# print("err = \n", err)
print("removed amounts = ", len(err))
data_ok = np.delete(data[:,:], err, axis=0).astype(np.float32);     # print(data_ok)；why這一次要加axis=0才能正常輸出??
print(data_ok[0:2,:])
np.savetxt(r'C:\Users\Chun-Po\Documents\Python Scripts\儲存分析結果(數據)\改掉SOH定義的資料集_noerror.csv',data_ok,fmt='%s',delimiter=',')

In [ ]:
'''各個特徵和soh的相關性'''

a = '改掉SOH定義的資料集_noerror.csv'
data_ok = np.genfromtxt(a, dtype = None, delimiter = ',', skip_header = 0, filling_values = 0, invalid_raise = False, encoding = 'UTF-8')   #'latin-1'
[Row,Column] = data_ok.shape
print("data_ok[0] = ", data_ok[:,0])    #type()
print("data_ok[1] = ", data_ok[:,1])    #type()

title = ['Total charging time','Relax 03min','inflection point','V15min','Dis1_Time','VPVD']
xterm = ['charging time(sec)','voltage variation(V)','charging time(sec)','voltage(V)','1st discharge time(sec)','voltage(V)']
R_noerr = [];
for i in range(1,7,1):
    R = np.corrcoef(data_ok[:,0].tolist(),data_ok[:,i].tolist())[0,1];   # ps:corrcoef裡面必須放list
    R_sq = np.round(R*R,3)
    R_noerr.append(R_sq)
  # 以下畫圖
    plt.figure(figsize=(5,5), dpi = 100)
    plt.plot(data_ok[:,i],data_ok[:,0], color='blue', linestyle='None', marker='.', markersize=1);
    m1, b1 = np.polyfit(data_ok[:,i],data_ok[:,0], 1);
    plt.axline(xy1=(0, b1), slope=m1, color='g', label=f'R^2 = {R_sq}')   #$y = {m1:.2f}x {b1:+.2f}$\n 
    plt.xlabel(xterm[i-1], fontsize=15)
    plt.ylabel("SOH (%)", fontsize=15)
    plt.title(title[i-1], fontsize=20);
    if i in range(4,7,2):
        plt.xlim(min(data_ok[:,i])-0.25,max(data_ok[:,i])+0.25);
    elif i == 1:
        plt.xlim(600,max(data_ok[:,i])+300);
    plt.ylim(min(data_ok[:,0])-5,max(data_ok[:,0])+5);
    plt.grid(color='r', linestyle='-.', linewidth=0.2)
    plt.legend(fontsize=15); plt.show()

print("R^2 of every feature=",R_noerr)

In [ ]:
# 算多少比例的電池沒有在DCD35下充飽
fullcd = np.where(data[:,1] > 3.788)[0]   # number of fullcd =  4619；   18% (2098秒以上)
#fullcd = np.where(data[:,1] > 3.9)[0]    # number of fullcd =   916；3.585% (2160秒以上)
#fullcd = np.where(data[:,1] > 4)[0]      # number of fullcd =   725；2.838% (2216秒以上)
#fullcd = np.where(data[:,1] > 4.004)[0]  # number of fullcd =   717；2.838% (2218秒以上)
print("position of fullcd = ", fullcd)
print("number of fullcd = ", len(fullcd))

# Gradient Boosting

In [ ]:
import time
t1 = time.monotonic()

from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

#X, y = make_regression(random_state=0)
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
reg = GradientBoostingRegressor(random_state=0)
reg.fit(features_train2, soh_train2)
prediction = reg.predict(features_test2)  # features_train2, features_test2, soh_train2, soh_test2
R_sq = reg.score(features_test2, soh_test2)

print("R_sq = ", R_sq)

t2 = time.monotonic()
print("time elapsed:" + str(np.round(t2-t1,3)) + " seconds")

In [ ]:
import time
t1 = time.monotonic()

rmseoos = []
maeoos = []
predvalue = []
'''超參數 (每次訓練完要複製)'''
variables = range(10, 410, 30)
for variable in variables:
    # nest = 200
    depth = 5
    learnrate = 0.1
    subsamp = 0.5
    model = ensemble.GradientBoostingRegressor(n_estimators=variable, max_depth=depth, learning_rate=learnrate, subsample = subsamp, loss='squared_error')
    model.fit(features_train, pred_soh_train)
    predictions = model.staged_predict(features_test)
    for p in predictions:
        predvalue.append(p)
        rmseoos.append(math.sqrt(mean_squared_error(pred_soh_test, p)))
        maeoos.append(mean_absolute_error(pred_soh_test, p))
R_sq = np.round(model.fit(features_train, pred_soh_train).score(features_test, pred_soh_test),3)
print("predvalue = ", predvalue)
print("   R_sq = ", R_sq)
print(" MAE(%) = ", np.round(maeoos[-1],3))
print("RMSE(%) = ", np.round(rmseoos[-1],3))
print("Lowest RMSE(%) = ", np.round(min(rmseoos),3))
print("Optimal place = ", rmseoos.index(min(rmseoos)))

t2 = time.monotonic()
print("time elapsed:" + str(np.round(t2-t1,3)) + " seconds")

In [ ]:
f = plot.figure(figsize=(12,6))
ax1 = f.add_subplot(121); ax2 = f.add_subplot(122)

ax1.plot(predvalue,pred_soh_test, 'b.')
m1, b1 = np.polyfit(predvalue,pred_soh_test, 1);
ax1.axline(xy1=(0, b1), slope=m1, color='g', label=f'R_sq   = {R_sq}\n   MAE = {MAE}%\nRMSE = {RMSE}%') # 
ax1.set_xlabel("Predicted SOH(%)", fontsize=15) 
ax1.set_ylabel("Real SOH(%)", fontsize=15)
ax1.legend(fontsize=15)

ax2.plot(variables, rmseoos)
ax2.set_xlabel("random_state_labels", fontsize=18) 
ax2.set_ylabel("RMSE(%)", fontsize=18)